In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime
import os

In [95]:
import validators

In [96]:
import warnings

In [97]:
from pprint import pprint

In [98]:
import plotly.express as px

In [99]:
import seaborn as sns

In [100]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [101]:
import glob

In [102]:
## SEC-Registered

In [103]:
df_sec = pd.read_csv(glob.glob('data/sec/IA_FIRM_SEC_Feed_*.csv')[0],
                     low_memory=False)

In [104]:
# sorted(df_sec.columns.to_list())

In [105]:
## State-Registered Advisers 

In [106]:
df_state = pd.read_csv(glob.glob('data/sec/IA_FIRM_STATE_Feed_*.csv')[0],
                       low_memory=False)

In [107]:
# sorted(df_state.columns.to_list())

In [108]:
## Merging SEC + State Registered RIA data 
### (N.B. some firms appear in both datasets, so must be fixed)

In [109]:
df_state_ids = df_state.loc[:,['Info.@BusNm',
                             'Info.@FirmCrdNb',
                             'Info.@LegalNm',
                             'Info.@SECNb']]

In [110]:
df_sec_ids = df_sec.loc[:,['Info.@BusNm',
                         'Info.@FirmCrdNb',
                         'Info.@LegalNm',
                         'Info.@SECNb']]

In [111]:
df_all_ids = pd.merge(df_state_ids, df_sec_ids,
                      how='outer',
                      # on=['Info.@FirmCrdNb', 'Info.@SECNb', 'Info.@BusNm', 'Info.@LegalNm'],
                      indicator=True)

In [112]:
### Rows

In [113]:
#### CRD numbers are unique identifiers within each dataset.

In [114]:
# df_state_ids.nunique()/df_state_ids.shape[0]

In [115]:
# df_sec_ids.nunique()/df_sec_ids.shape[0]

In [116]:
# number_of_firms_state_and_sec = df_all_ids[df_all_ids['_merge'] == 'both'].shape[0]
# print(f'{number_of_firms_state_and_sec} firms appear in both the State and the SEC datasets')
# print(f'{number_of_firms_state_and_sec/df_all_ids.shape[0]:.2%} of total')

In [117]:
#### TODO: we must clean this up

In [118]:
### Columns

In [119]:
#### The majority of columns are common to both state & SEC registered, but there are significant differences in the set of columns

In [120]:
df_state_cols = set(df_state.columns)
df_sec_cols = set(df_sec.columns)

In [121]:
# print('There are %s unique columns in total' %len(df_sec_cols.union(df_state_cols)))
# print('')
# print('%s columns are in both State & SEC' %len(df_sec_cols.intersection(df_state_cols)))
# print('')
# print('%s columns are in only one of State or SEC' %len(df_sec_cols.symmetric_difference(df_state_cols)))
# print('of which:')
# print('%s columns are in only SEC' %len(df_sec_cols.difference(df_state_cols)))
# print('%s columns are in only State' %len(df_state_cols.difference(df_sec_cols)))

In [122]:
### Clean up the data

In [123]:
# We'll process the data in 3 parts:
# 1. Firms in both (the hard part, as entries for common columns may not be the same across the 2 datasets)
# 2. Firms in State only
# 3. Firms in SEC only

In [124]:
df_both_ids = df_all_ids[df_all_ids._merge == 'both'].drop(columns=['_merge'])
df_state_only_ids = df_all_ids[df_all_ids._merge == 'left_only'].drop(columns=['_merge'])
df_sec_only_ids = df_all_ids[df_all_ids._merge == 'right_only'].drop(columns=['_merge'])

In [125]:
# df_both_ids.shape[0]

In [126]:
df_both_state_data = pd.merge(df_state, df_both_ids,
                              how = 'right',
                              on=['Info.@FirmCrdNb', 'Info.@SECNb', 'Info.@BusNm', 'Info.@LegalNm'])

In [127]:
df_both_sec_data = pd.merge(df_sec, df_both_ids,
                            how = 'right',
                            on=['Info.@FirmCrdNb', 'Info.@SECNb', 'Info.@BusNm', 'Info.@LegalNm'])

In [128]:
# this operation can cause a lot of "type mismatch" style errors
# TODO: specify dtype on the initial pd.read_csv methods & confirm this fixes the errors
with warnings.catch_warnings():
    warnings.simplefilter(action='ignore', category=FutureWarning)
    df_both = pd.merge(df_both_state_data, df_both_sec_data,
                       how='outer',
                       suffixes = ('__State', '__SEC'))

In [129]:
# df_both.shape[0]

In [130]:
# We gained extra rows during merge -- so some data in the common columns did not match

In [131]:
df_both_value_counts = df_both.value_counts(subset=['Info.@FirmCrdNb', 'Info.@SECNb', 'Info.@BusNm', 'Info.@LegalNm'])

In [132]:
df_both_mismatched_data_ids = df_both_value_counts[df_both_value_counts > 1].index.to_frame().reset_index(drop=True)

In [133]:
# df_both_mismatched_data_ids

In [134]:
merged_series = []
for row in df_both_mismatched_data_ids.iterrows():
    df_mismatch = df_both[(df_both['Info.@FirmCrdNb'] == row[1]['Info.@FirmCrdNb']) &\
                           (df_both['Info.@SECNb'] == row[1]['Info.@SECNb']) &\
                           (df_both['Info.@BusNm'] == row[1]['Info.@BusNm']) &\
                           (df_both['Info.@LegalNm'] == row[1]['Info.@LegalNm'])].copy()
    df_both.drop(index=df_mismatch.index, inplace=True)
    df_mismatch = df_mismatch.transpose()
    increasing_nulls_index = df_mismatch.isna().sum().sort_values(ascending=True).index
    best_merge = df_mismatch.loc[:,increasing_nulls_index[0]].combine_first(df_mismatch.loc[:,increasing_nulls_index[1]])
    merged_series.append(best_merge)

In [135]:
df_both_mismatched_data_fixed = pd.DataFrame(merged_series)

In [136]:
df_both = pd.concat([df_both, df_both_mismatched_data_fixed]).reset_index(drop=True)

In [137]:
# df_both

In [138]:
df_only_state_data = pd.merge(df_state, df_state_only_ids,
                              how = 'right',
                              on=['Info.@FirmCrdNb', 'Info.@SECNb', 'Info.@BusNm', 'Info.@LegalNm'])

In [139]:
df_only_sec_data = pd.merge(df_sec, df_sec_only_ids,
                            how = 'right',
                            on=['Info.@FirmCrdNb', 'Info.@SECNb', 'Info.@BusNm', 'Info.@LegalNm'])

In [140]:
df_state_or_sec = pd.merge(df_only_state_data, df_only_sec_data,
                   how='outer',
                   suffixes = ('__State', '__SEC'))

In [141]:
assert df_state_or_sec.columns.str.endswith('__State').any() == False
assert df_state_or_sec.columns.str.endswith('__SEC').any() == False

In [142]:
df_all = pd.merge(df_state_or_sec, df_both,
                   how='outer',
                   suffixes = ('__State', '__SEC'))

In [143]:
# df_all

In [144]:
assert df_all.shape[0] == df_all_ids.shape[0]

In [145]:
# df_all.head()

In [146]:
# Get All Web Addresses

In [147]:
## Best not to use eval on data from the internet
## https://xkcd.com/327/

# def get_all_urls(df, valid_only=True):
#     df_urls = df.loc[df['FormInfo.Part1A.Item1.WebAddrs.WebAddr'].notnull(),
#                     ['Info.@FirmCrdNb',
#                      'FormInfo.Part1A.Item1.WebAddrs.WebAddr']
#                     ]
#     df_urls.rename(columns={'FormInfo.Part1A.Item1.WebAddrs.WebAddr':'url'}, inplace=True)
#     df_urls['url'] = df_urls['url'].apply(lambda x: eval(x.lower()) if ('[' in x and ']' in x) else x.lower())
#     df_urls = df_urls.explode('url').reset_index(drop=True)
#     df_urls['url_valid'] = df_urls['url'].apply(validators.url)
#     if valid_only:
#         df_urls = df_urls.loc[df_urls['url_valid'] == True,:]
#     return df_urls

In [148]:
from urllib.parse import urlparse
from collections import defaultdict

In [149]:
def parse_array(web_addr):
    if web_addr in (None, np.NaN):
        return np.NaN
    web_addr = web_addr.lower()
    if web_addr.startswith("["):
        web_addr = web_addr.replace("'", '"')
        web_addr = web_addr.removeprefix("[").removesuffix("]")
        candidates = [url.removeprefix('"').removesuffix('"') for url in web_addr.split(", ")]
    else:
        candidates = [web_addr]
    return candidates

In [150]:
df_all['urls'] = df_all['FormInfo.Part1A.Item1.WebAddrs.WebAddr'].apply(parse_array)

In [151]:
# df_all['urls']

In [152]:
def parse_urls(urls):
    if not isinstance(urls, list):
        return np.NaN, np.NaN
    domains = []
    schemes = []
    for url in urls:
        if validators.url(url):
            parsed_url = urlparse(url)
            domain = parsed_url.netloc
            if domain.startswith("www."):
                domain = domain.removeprefix("www.")
            elif domain.startswith("m."):
                domain = domain.removeprefix("m.")
            elif domain.startswith("mobile."):
                domain = domain.removeprefix("mobile.")
            domains.append(domain)
            schemes.append(parsed_url.scheme)
    return domains, schemes

In [153]:
# TODO: make less fugly
df_all_urls = df_all.apply(lambda x: parse_urls(x['urls']), axis=1, result_type='expand')
df_all_urls.columns = ['url_domains', 'url_schemes']

In [154]:
df_all = df_all.join(df_all_urls)

In [155]:
# df_all[['Info.@FirmCrdNb', 'Info.@SECNb', 'Info.@BusNm', 'Info.@LegalNm', 'urls','url_domains', 'url_schemes']].head()

In [156]:
SOCIAL_MEDIA = [
    'linkedin.com',
    'facebook.com',
    'twitter.com',
    'instagram.com',
    'youtube.com',
    'vimeo.com',
    'medium.com',
    'tiktok.com',
    'pinterest.com',
    'soundcloud.com',
    'plus.google.com',
    'open.spotify.com',
    'reddit.com',
    'linkedin',
    'podcasts.apple.com',
    'seekingalpha.com',
    'apple.co', #TODO: Prcessing error? #podcasts.apple.com
    'apple.com',
    'glassdoor.com',
    'spoti.fi',
    'ca.linkedin.com',
    'business.facebook.com',
    'mp.weixin.qq.com', #wechat
    'hvst.com',
    'douyin.com', #tiktok in chinese
    'anchor.fm', #podcasts
    'line.naver.jp', #chinese wechat competitor, only Blackrock is on it!
    'hk.linkedin.com', 
    'quora.com', 
    'slideshare.net', #sharing ppt's!
    'g.page',  #google business page
    'sg.linkedin.com',
    'au.linkedin.com',  
    'stitcher.com', # podcasts
    'itunes.apple.com', # podcasts
    'instragram.com', #typo, but it still redirects
    'podcasts.google.com',   
    'uk.linkedin.com', 
    'facebook', #TODO: Prcessing error?
    'buzzsprout.com', #podcast
    'ja-jp.facebook.com', 
    'stocktwits.com',
    'blog.naver.com', #chinese wechat competitor only AB on this!
]

In [157]:
AGGREGATOR_SITES = [
    'yelp.com',
    'angel.co',
    'feeonlynetwork.com',
    'crunchbase.com',
    'thefinancialhq.com', #this is a weird one - no "homepage" to speak of
    'napfa.org',
    'alignable.com',
    'realwealthmedia.com',
    'letsmakeaplan.org' # CFP official site (for clients)
    'google.com', # Typically google maps
    'nextdoor.com',
    'xyplanningnetwork.com',
    'plannersearch.org', # FPA official website
]

In [158]:
LINKEDIN_SITES = [
                    'linkedin.com',
                    'linkedin',
                    'ca.linkedin.com',
                    'hk.linkedin.com', 
                    'sg.linkedin.com',
                    'au.linkedin.com', 
                    'uk.linkedin.com', 
]

In [159]:
processing_errors = [
    'apple.co',
    'link.edgepilot.com',
    'facebook',
]

In [160]:
def isin_group(x, group_list, return_bool=True, handle_nulls = True, inverse=False):
    if handle_nulls and isinstance(x, float): # np.NaN is of type float!
        if np.isnan(x): # but the floats actually have to be eaual to nan
            if return_bool:
                return False
            return np.NaN
    if return_bool:
        if inverse:
            return any([(el not in group_list) for el in x if el is not None])
        return any([(el in group_list) for el in x if el is not None])
    else:
        if inverse:
            return [el for el in x if (el not in group_list) and (el is not None)]
        return [el for el in x if (el in group_list) and (el is not None)]

In [161]:
# def notin_group(x, group_list, return_bool=True, handle_nulls = True, inverse=False):
#     if handle_nulls and isinstance(x, float): 
#         if np.isnan(x): 
#             if return_bool:
#                 return False
#             return np.NaN
#     if return_bool:
#         return_bool = any([(el not in group_list) for el in x if el is not None])
#         if inverse:
#             return not return_bool
#         return return_bool
#     else:
#         if inverse:
#             return [el for el in x if (el in group_list) and (el is not None)]
#         return [el for el in x if (el not in group_list) and (el is not None)]

In [162]:
df_all['url_has_linkedin'] = df_all['url_domains'].apply(isin_group, group_list=LINKEDIN_SITES)
df_all['url_has_social_media'] = df_all['url_domains'].apply(isin_group, group_list=SOCIAL_MEDIA)
df_all['url_has_aggregator'] = df_all['url_domains'].apply(isin_group, group_list=AGGREGATOR_SITES)
df_all['url_has_firm_specific'] = df_all['url_domains'].apply(isin_group, group_list=SOCIAL_MEDIA + AGGREGATOR_SITES, inverse=True)

df_all['url_social_media'] = df_all['url_domains'].apply(isin_group, group_list=SOCIAL_MEDIA, return_bool=False)
df_all['url_aggregator'] = df_all['url_domains'].apply(isin_group, group_list=AGGREGATOR_SITES, return_bool=False)
df_all['url_firm_specific'] = df_all['url_domains'].apply(isin_group, group_list=SOCIAL_MEDIA + AGGREGATOR_SITES, inverse=True, return_bool=False)

In [163]:
df_all['url_social_media_count'] = df_all['url_social_media'].apply(lambda x: len(set(x)) if isinstance(x, list) else 0)

In [164]:
# df_all['url_social_media_count'].max()

In [165]:
# df_all.head()

In [166]:
# Summarize Firm AUM values

In [167]:
df_all['TotalAUM'] = df_all.loc[:,(df_all.columns.str.startswith('FormInfo.Part1A.Item5D') \
                                   & df_all.columns.str.endswith('3'))           
                               ].sum(axis=1)

In [168]:
## Firms by Registration Type

In [169]:
df_both_ids['registration'] = 'both'
df_state_only_ids['registration'] = 'state'
df_sec_only_ids['registration'] = 'sec'

In [170]:
df_registration = pd.concat([df_both_ids, df_state_only_ids, df_sec_only_ids])

In [171]:
df_all = df_all.merge(df_registration, how='left')

In [172]:
## Identify ERAs (so we can ignore them)

In [173]:
df_era_ids = df_all.loc[(df_all['ERA.Rgltrs'].notnull() |
                         df_all['ERA.Rgltrs.Rgltr'].notnull() |
                         df_all['ERA.Rgltrs.Rgltr.@Cd'].notnull() |
                         df_all['ERA.Rgltrs.Rgltr.@Dt'].notnull() |
                         df_all['ERA.Rgltrs.Rgltr.@St'].notnull() |
                         ( df_all['Rgstn.@FirmType'] == 'ERA' ) ),
                        'Info.@FirmCrdNb']

df_all.loc[df_era_ids.index, 'ERA'] = True
df_all['ERA'] = df_all['ERA'].fillna(False)

In [174]:
# recode zero employees
df_all['TotalEmp'] = np.where(df_all['FormInfo.Part1A.Item5A.@TtlEmp'] > 0, df_all['FormInfo.Part1A.Item5A.@TtlEmp'], 1)

In [175]:
df_ria = df_all[df_all['ERA'] == False]

In [176]:
if not os.path.exists('data/clean'):
    os.makedirs('data/clean')

In [177]:
df_all.to_csv('data/clean/ALL_03_20_2023.csv', index=False)
df_ria.to_csv('data/clean/RIA_03_20_2023.csv', index=False)